In [ ]:
# ============================================================================
# 2 Fitts' Law
# ---------------------------------------
# GUI Usage:
#   0) For better effect, please cover jupyter notebook inline output when doing experiment in GUI
#   1) Please CONCENTRATE on pointing between two buttons, and clicking on the button when you point to it
#   2) Sliders: to change the width of / distance between two objects
#   3) ID: real-time calcularion of Index of Difficulty according to width and distance
#   4) Count down times: to indicate how many times left for finishing this round of pointings
#   5) To get polynominal data fitting, you need to try at least 3 ID settings
# Preassumption:
#   1) Moving locus is 1D horizontal straight line
#   2) Start timing after tapping on either one of the two buttons
# ============================================================================

import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import time as tm
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import numpy as np
import string

counter = 0
counterMax = 1
counterId = 0   # count the number of ID settings, for data fitting
timeStamp = 0
timeStampTemp = 0
timeDelta = 0
areaWidth = 700
widthValue = 40
distanceValue = 250
idValue = 4.0
areaCornerWidth = (areaWidth - widthValue*2 - distanceValue)/2
idList = []     # list of Index of Difficulty
mtList = []     # list of Movement Time
z = []          # list of polynominal coefficients

def onScaleChanged(event):
    global widthValue
    global distanceValue
    global areaCornerWidth
    global idValue
    global idList
    # change the width of objects
    widthValue = int(varWidth.get())
    frameObj1.config(width=widthValue)
    frameObj2.config(width=widthValue)
    # change the distance between objects
    distanceValue = varDistance.get()
    areaCornerWidth = (areaWidth - widthValue*2 - distanceValue)/2
    frameDistance.config(width=distanceValue)
    frameNWCorner.config(width=areaCornerWidth)
    frameSWCorner.config(width=areaCornerWidth)
    frameNECorner.config(width=areaCornerWidth)
    frameSECorner.config(width=areaCornerWidth)
    idValue = np.log2(2*distanceValue/widthValue)
    labelId.config(text="ID = %0.2f" %idValue)

# limit counter entry to be only digits
def onValidate(d, S):
    if S in string.digits:
        return True
    else:
        return False

def onStartClicked(event):
    if not entryCounter.get():
        tk.messagebox.showinfo("Reminder", "Please input counter.")
    else:
        buttonStart.grid_forget()
        buttonStop.grid(row=0, column=0, padx=10, pady=10, ipadx=10, sticky=tk.NSEW)
        global counter
        counter = 0
        global counterMax
        counterMax = int(entryCounter.get())
        countDown.set(counterMax)
        labelCountDownText.grid(row=1, column=4, padx=10, sticky=tk.SW)
        labelCountDown.grid(row=2, column=4, padx=10, sticky=tk.NSEW)
        scaleWidth.config(state=tk.DISABLED)
        scaleDistance.config(state=tk.DISABLED)
        entryCounter.config(state=tk.DISABLED)
        obj1.config(bg='#9ED2CC', state=tk.NORMAL)
        obj2.config(bg='#9ED2CC', state=tk.NORMAL)
        obj1.bind("<ButtonRelease-1>", cursorMovement)
        obj1.bind("<Button-1>", onCounter)
        obj2.bind("<ButtonRelease-1>", cursorMovement)
        obj2.bind("<Button-1>", onCounter)

def onStopClicked(event):
    buttonStart.grid(row=1, column=0, padx=10, pady=10, rowspan=3, ipadx=10, sticky=tk.NSEW)
    buttonStop.grid_forget()
    global counter
    counter = 0
    scaleWidth.config(state=tk.NORMAL)
    scaleDistance.config(state=tk.NORMAL)
    entryCounter.config(state=tk.NORMAL)
    labelCountDownText.grid_forget()
    labelCountDown.grid_forget()

def onCounter(event):
    global counter
    if counter <= counterMax:
        counter += 1
        countDown.set(counterMax - counter + 1)

def cursorMovement(event):
    global timeStamp
    global timeStampTemp
    global counter
    global counterId
    global idList
    global mtList
    if counter == 1:
        timeStampTemp = tm.clock()
    if counter >= 1 and counter <= counterMax:
        # avoid clicking on the same object twice
        if event.widget == obj1:
            obj1.config(state=tk.DISABLED, bg="SystemButtonFace")
            obj1.unbind("<ButtonRelease-1>")
            obj1.unbind("<Button-1>")
            obj2.config(state=tk.NORMAL, bg='#9ED2CC')
            obj2.bind("<ButtonRelease-1>", cursorMovement)
            obj2.bind("<Button-1>", onCounter)
        if event.widget == obj2:
            obj1.config(state=tk.NORMAL, bg='#9ED2CC')
            obj1.bind("<ButtonRelease-1>", cursorMovement)
            obj1.bind("<Button-1>", onCounter)
            obj2.config(state=tk.DISABLED, bg="SystemButtonFace")
            obj2.unbind("<ButtonRelease-1>")
            obj2.unbind("<Button-1>")
        if counter >=2:
            # timing cursor one-way movement time in millisecond
            timeStamp = tm.clock()
            timeDelta = (timeStamp - timeStampTemp)*1000
            timeStampTemp = timeStamp
            mtList.append(timeDelta)
            tk.Label(frameTime, text="time = %0.2f ms" %mtList[-1], anchor = tk.W).grid(row = 1 + counter, column=0, sticky=tk.N)
            idList.append(idValue)
            # plot movement time against index of Difficulty in real-time
            fig = plt.figure()
            graph = fig.add_subplot(111)
            graph.set(title="Fitts' Law Empirical Data Fitting", ylabel='Movement Time (millisecond)', xlabel='Index of Difficulty (ID) = $log_2 \\frac{2D}{W}$')
            graph.scatter(idList, mtList, color='#4A857E', alpha=0.5)
            if counterId >= 2:
                # polynomial data fitting
                global z
                z = np.polyfit(idList, mtList, 1)
                f = np.poly1d(z)
                idArray = np.array(idList)
                x = np.arange(idArray.min(),idArray.max(), 0.1)
                y = f(x)
                plt.plot(x, y)
            display.clear_output(wait=True)
            plt.grid()
            plt.show()
    if counter == counterMax:
        counterId += 1
    if counter > counterMax:
        obj1.config(state=tk.DISABLED, bg="SystemButtonFace")
        obj2.config(state=tk.DISABLED, bg="SystemButtonFace")
        scaleWidth.config(state=tk.NORMAL)
        scaleDistance.config(state=tk.NORMAL)
        buttonStop.grid_forget()
        buttonStart.grid(row=1, column=0, padx=10, pady=10, rowspan=3, ipadx=10, sticky=tk.NSEW)

def onClearDataClicked(event):
    global idList
    idList = []
    global mtList
    mtList = []
    # delete / redraw plot
    display.clear_output(wait=True)
    print ("Clear data, try again!")

root = tk.Tk()
root.title("Fitts' Law: Empirical Experiment GUI")
root.resizable(width=False, height=False)
#experiment GUI
frameIdController = tk.Frame(root, width=areaWidth, height=100)
framePointingArea = tk.Frame(root, width=areaWidth, height=250, borderwidth=2, relief=tk.GROOVE)
frameTime = ttk.Labelframe(root, text='Delta Time')
frameIdController.grid(row=0, column=0, padx=10, pady=10, ipadx=10, ipady=10, sticky=tk.EW)
framePointingArea.grid(row=1, column=0, padx=10, pady=10, ipadx=10, ipady=10)
framePointingArea.grid_propagate(0)

# width, distance controller
varWidth = tk.DoubleVar()
varWidth.set(widthValue)
varDistance = tk.DoubleVar()
varDistance.set(distanceValue)
scaleWidth = tk.Scale(frameIdController, from_=10, to=70, variable=varWidth, command=onScaleChanged, orient=tk.HORIZONTAL)
scaleDistance = tk.Scale(frameIdController, from_=50, to=500, variable=varDistance, command=onScaleChanged, orient=tk.HORIZONTAL)
vcmd = (frameIdController.register(onValidate), '%d', '%S')
textEntry = tk.StringVar()
textEntry.set("10")
entryCounter = tk.Entry(frameIdController, textvariable=textEntry, validate="key", validatecommand=vcmd)
countDown = tk.IntVar()
labelCountDown = tk.Label(frameIdController, textvariable=countDown, bg='white', fg='black', font=("Arial", 20), width=9)
labelCountDownText = tk.Label(frameIdController, text="Remaining clicking(s):")
labelWidth = tk.Label(frameIdController, text="Width (pixel)")
labelDistance = tk.Label(frameIdController, text="Distance (pixel)")
labelId = tk.Label(frameIdController, text="ID = %0.2f" %idValue, bg='#9ED2CC', width=10, font=("Arial", 20))
labelCounter = tk.Label(frameIdController, text="Counter =")
labelCounterSuggestion = tk.Label(frameIdController, text="(Suggested counter: 10 - 20)")
buttonStart = tk.Button(frameIdController, text="Start", width=10, padx=10, pady=10)
buttonStop = tk.Button(frameIdController, text="Stop", width=10, padx=10, pady=10)
buttonClearData = tk.Button(frameIdController, text="Clear Data", width=20, padx=10, pady=10)

buttonStart.grid(row=1, column=0, padx=10, pady=10, rowspan=3, ipadx=10, sticky=tk.NSEW)
labelWidth.grid(row=0, column=1, sticky=tk.W+tk.S)
scaleWidth.grid(row=0, column=2, sticky=tk.EW+tk.S)
labelDistance.grid(row=1, column=1, sticky=tk.W+tk.S)
scaleDistance.grid(row=1, column=2, sticky=tk.EW+tk.N)
labelCounter.grid(row=2, column=1, pady=20, sticky=tk.W+tk.S)
entryCounter.grid(row=2, column=2, pady=20, sticky=tk.EW+tk.S)
labelCounterSuggestion.grid(row=3, column=2, sticky=tk.NE)
labelId.grid(row=0, column=3, rowspan=4, padx=10, pady=10, sticky=tk.NSEW)
labelId.grid_propagate(0)
buttonClearData.grid(row=0, column=4, padx=10, pady=10, sticky=tk.NSEW)

# Pointing area
frameNWCorner = tk.Frame(framePointingArea, width=areaCornerWidth, height=100)
frameSWCorner = tk.Frame(framePointingArea, width=areaCornerWidth, height=100)
frameNECorner = tk.Frame(framePointingArea, width=areaCornerWidth, height=100)
frameSECorner = tk.Frame(framePointingArea, width=areaCornerWidth, height=100)
frameDistance = tk.Frame(framePointingArea, width=distanceValue, height=50)
frameObj1 = tk.Frame(framePointingArea, width=widthValue, height=50)
frameObj2 = tk.Frame(framePointingArea, width=widthValue, height=50)
obj1=tk.Button(frameObj1, state=tk.DISABLED, activebackground='#4A857E')
obj2=tk.Button(frameObj2, state=tk.DISABLED, activebackground='#4A857E')
frameNWCorner.grid(row=0, column=0)
frameNECorner.grid(row=0, column=4)
frameObj1.grid(row=1, column=1)
frameObj1.rowconfigure(0, weight=1)
frameObj1.columnconfigure(0, weight=1)
frameObj1.grid_propagate(0)
obj1.grid(sticky=tk.NSEW)
frameDistance.grid(row=1, column=2)
frameDistance.grid_propagate(0)
frameObj2.grid(row=1, column=3)
frameObj2.rowconfigure(0, weight=1)
frameObj2.columnconfigure(0, weight=1)
frameObj2.grid_propagate(0)
obj2.grid(sticky=tk.NSEW)
frameSWCorner.grid(row=2, column=0)
frameSECorner.grid(row=2, column=4)

buttonStart.bind("<ButtonRelease-1>", onStartClicked)
buttonStop.bind("<ButtonRelease-1>", onStopClicked)
buttonClearData.bind("<ButtonRelease-1>", onClearDataClicked)

root.mainloop()


In [ ]:
# After you get polynominal data fitting from the 1st cell, run this cell to see your modelling result.
print ("Empirical data fitting result:\nMT = %0.2f + %0.2f ID" %(z[0], z[1]))

In [ ]:
# (Optional) If you have further interest, try to plot the ratio of Movement Time and Index of Difficulty!
# What can you find?
fig2 = plt.figure()
graph = fig2.add_subplot(111)
graph.set(title="What can you find?", ylabel='Ratio of MT (ms) and ID', xlabel='No. of Trials')
number = []
ratio = []
for i in range(len(mtList)):
    mtValue = mtList[i]
    idValue = idList[i]
    ratio.append(mtList[i]/idList[i])
    number.append(i)
graph.scatter(number, ratio, color='#4A857E', alpha=0.7)
plt.grid()
plt.show()
# The ratio tend to be constant (might be slightly decreased as trails going), could be interpreted as an Index of Performance,
# which indicates the capacity of human motor system, like the capacity of communication channel in Shannon information theory.